In [27]:
pip install keras-tuner


In [29]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [30]:
fashion_mnist = keras.datasets.fashion_mnist

In [31]:
(train_images, train_labels),(test_images, test_labels)=fashion_mnist.load_data()

In [32]:
train_images = train_images/255.0 
test_images = test_images/255.0

In [33]:
train_images[0].shape

(28, 28)

In [34]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images  = test_images.reshape(len(test_images), 28,28,1)

In [58]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model
  

In [47]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [60]:
tuner_search=RandomSearch(build_model,objective='val_accuracy', max_trials=5,directory='output',project_name="mnist fashion")

In [61]:
tuner_search.search(train_images,train_labels,epochs = 3 ,validation_split =0.1)


Trial 5 Complete [00h 13m 08s]
val_accuracy: 0.9073333144187927

Best val_accuracy So Far: 0.9073333144187927
Total elapsed time: 00h 49m 31s
INFO:tensorflow:Oracle triggered exit


In [62]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 128)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        73792     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3469424   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,547,674
Trainable params: 3,547,674
Non-trainable params: 0
_________________________________________________________________
